In [ ]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

If you are using Google Colab, go to data directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks/2019TWBootcamp/

## Parameters

In [ ]:
import pathlib
#img_path = "/home/Data/CharactersTrimPad28/"
#img_path = "./s3mnt/ChineseNumbers/"
img_path = "/home/Data/ChineseNumbers/"
data_root = pathlib.Path(img_path)
if not data_root.exists():
    print("{} not exist!".format(data_root))

AUTOTUNE = tf.data.experimental.AUTOTUNE
NUM_EPOCHS = 3
BATCH_SIZE = 256 
IMG_SIZE = 32

## Generate filename and label list

In [ ]:
# tf.data.Dataset.from_tensor_slices
all_image_paths = [str(path) for path in list(data_root.glob('*/*'))]

label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

image_count = len(all_image_paths) * NUM_EPOCHS

for i in range(3):
    print(all_image_paths[i])
image_count

## Mapping function

In [ ]:
def preprocess_image(image_str):
    image = tf.image.decode_png(image_str, channels=3)
    image = tf.image.resize_images(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32)
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path):
    image_str = tf.read_file(path)
    return preprocess_image(image_str)

# The tuples are unpacked into the positional arguments of the mapped function
def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

## Testing Model

In [ ]:
input_shape = (IMG_SIZE, IMG_SIZE, 3)
num_class = len(label_names)

### Test with VGG16

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(
                input_shape=input_shape,
                include_top=False,
                weights='imagenet')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_class, activation='softmax')

model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

### Test with MLP

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

### Print Model Summary

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
             )
model.summary()

### Test iterate time

In [ ]:
import time

def timeit(dataset):
    overall_start = time.time()
    n_image = 0
    start = time.time()
    
    history = model.fit(dataset,
                        epochs=1)
    ''' 
    for n_batch, (images, labels) in enumerate(dataset):
        if n_batch%10 == 0:
            print("\r{} images: {:.2f} s".format(n_batch * BATCH_SIZE, time.time()-start), 
                                                 end='', flush=True)
    ''' 
    end = time.time()
    duration = end-start
    
    print("{} images: {:0.2f} s".format(image_count, duration))
    print("{:0.5f} Images/s".format(image_count/float(duration)))

# Input pipeline experiment

## 1. Original pipeline

In [19]:
# Extract
path_label_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
path_label_ds = path_label_ds.shuffle(buffer_size=image_count)
path_label_ds = path_label_ds.repeat(NUM_EPOCHS)

image_label_ds = path_label_ds.map(load_and_preprocess_from_path_label)
image_label_ds = image_label_ds.batch(BATCH_SIZE)

# Load
timeit(image_label_ds)

 74/148 [==============>...............] - ETA: 5:51 - loss: 107.1452 - acc: 0.0476

KeyboardInterrupt: 

## 2. Prefetch

See more about [tf.data.experimental.prefetch_to_device](https://www.tensorflow.org/api_docs/python/tf/data/experimental/prefetch_to_device)
```
tf.data.experimental.prefetch_to_device(
    device,
    buffer_size=None
)
```

In [ ]:
# Extract
path_label_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
path_label_ds = path_label_ds.shuffle(buffer_size=image_count)
path_label_ds = path_label_ds.repeat(NUM_EPOCHS)

image_label_ds = path_label_ds.map(load_and_preprocess_from_path_label)
image_label_ds = image_label_ds.batch(BATCH_SIZE)

# Load
# Prefetch must be final Dataset in input pipeline
image_label_ds = image_label_ds.prefetch(buffer_size=AUTOTUNE) # Only on CPU

timeit(image_label_ds)

In [ ]:
# Extract
path_label_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
path_label_ds = path_label_ds.shuffle(buffer_size=image_count)
path_label_ds = path_label_ds.repeat(NUM_EPOCHS)

#image_label_ds = path_label_ds.map(load_and_preprocess_from_path_label)
image_label_ds = path_label_ds.map(load_and_preprocess_from_path_label, num_parallel_calls=4)
image_label_ds = image_label_ds.batch(BATCH_SIZE)

# Load
# Prefetch must be final Dataset in input pipeline
image_label_ds = image_label_ds.apply(
    tf.data.experimental.prefetch_to_device(device="/gpu:0", buffer_size=AUTOTUNE)) 

timeit(image_label_ds)

## 3. Map with num_parallel_calls

In [ ]:
# Extract
path_label_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
path_label_ds = path_label_ds.shuffle(buffer_size=image_count)
path_label_ds = path_label_ds.repeat(NUM_EPOCHS)

image_label_ds = path_label_ds.map(load_and_preprocess_from_path_label, num_parallel_calls=4)
image_label_ds = image_label_ds.batch(BATCH_SIZE)

# Load
# Prefetch must be final Dataset in input pipeline
image_label_ds = image_label_ds.apply(
    tf.data.experimental.prefetch_to_device(device="/gpu:0", buffer_size=AUTOTUNE)) 

timeit(image_label_ds)

## 4. tf.data.experimental.shuffle_and_repeat

See more about [tf.data.experimental.shuffle_and_repeat](https://www.tensorflow.org/api_docs/python/tf/data/experimental/shuffle_and_repeat)
```
tf.data.experimental.shuffle_and_repeat(
    buffer_size,
    count=None,
    seed=None
)
```

In [ ]:
# Extract
path_label_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
#path_label_ds = path_label_ds.shuffle(buffer_size=image_count)
#path_label_ds = path_label_ds.repeat(NUM_EPOCHS)
#path_label_ds = path_label_ds.shuffle(buffer_size=image_count).repeat(NUM_EPOCHS)
path_label_ds = path_label_ds.apply(
    tf.data.experimental.shuffle_and_repeat(buffer_size=image_count, count=NUM_EPOCHS))

image_label_ds = path_label_ds.map(load_and_preprocess_from_path_label, num_parallel_calls=AUTOTUNE)
image_label_ds = image_label_ds.batch(BATCH_SIZE)

# Load
# Prefetch must be final Dataset in input pipeline
image_label_ds = image_label_ds.apply(
    tf.data.experimental.prefetch_to_device(device="/gpu:0", buffer_size=AUTOTUNE)) 

timeit(image_label_ds)

## 5. tf.data.experimental.map_and_batch

See more about [tf.data.experimental.map_and_batch](https://www.tensorflow.org/api_docs/python/tf/data/experimental/map_and_batch)
```
tf.data.experimental.map_and_batch(
    map_func,
    batch_size,
    num_parallel_batches=None,
    drop_remainder=False,
    num_parallel_calls=None
)
```

In [ ]:
# Extract
path_label_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
path_label_ds = path_label_ds.shuffle(buffer_size=image_count)
path_label_ds = path_label_ds.repeat(NUM_EPOCHS)

#image_label_ds = path_label_ds.map(load_and_preprocess_from_path_label, num_parallel_calls=AUTOTUNE)
#image_label_ds = image_label_ds.batch(BATCH_SIZE)
image_label_ds = path_label_ds.apply(
    tf.data.experimental.map_and_batch(load_and_preprocess_from_path_label, BATCH_SIZE, num_parallel_calls=AUTOTUNE))

# Load
# Prefetch must be final Dataset in input pipeline
image_label_ds = image_label_ds.apply(
    tf.data.experimental.prefetch_to_device(device="/gpu:0", buffer_size=AUTOTUNE)) 

timeit(image_label_ds)

## 6. shuffle_and_repeat + map_and_batch

In [ ]:
# Extract
path_label_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
path_label_ds = path_label_ds.apply(
    tf.data.experimental.shuffle_and_repeat(buffer_size=image_count, count=NUM_EPOCHS))
image_label_ds = path_label_ds.apply(
    tf.data.experimental.map_and_batch(load_and_preprocess_from_path_label, BATCH_SIZE, num_parallel_calls=AUTOTUNE))

# Load
# Prefetch must be final Dataset in input pipeline
image_label_ds = image_label_ds.apply(
    tf.data.experimental.prefetch_to_device(device="/gpu:0", buffer_size=AUTOTUNE)) 

timeit(image_label_ds)

## 7. Cache

See more about [tf.data.Dataset.cache](https://www.tensorflow.org/tutorials/load_data/images#cache)
                                       
Use tf.data.Dataset.cache to easily cache calculations across epochs. This is especially performant if the dataq fits in memory
```
ds = image_label_ds.cache()
```

One disadvantage to using an in memory cache is that the cache must be rebuilt on each run, giving the same startup delay each time the dataset is started:
If the data doesn't fit in memory, use a cache file. 
The cache file also has the advantage that it can be used to quickly restart the dataset without rebuilding the cache. Note how much faster it is the second time:


```
ds = image_label_ds.cache(filename='./cache.tf-data')
```

In [ ]:
# Extract
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
ds = ds.apply(
    tf.data.experimental.shuffle_and_repeat(buffer_size=image_count, count=NUM_EPOCHS))
ds = ds.apply(
    tf.data.experimental.map_and_batch(load_and_preprocess_from_path_label, BATCH_SIZE, num_parallel_calls=AUTOTUNE))

# Load
ds = ds.cache(filename='./cache.tf-ds')

# Prefetch must be final Dataset in input pipeline
ds = ds.apply(
    tf.data.experimental.prefetch_to_device(device="/gpu:0", buffer_size=AUTOTUNE)) 

timeit(ds)

In [ ]:
# Extract
path_label_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Transform
path_label_ds = path_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=image_count, count=NUM_EPOCHS))
path_label_ds = path_label_ds.cache(filename='./cache.tf-path')

image_label_ds = path_label_ds.map(load_and_preprocess_from_path_label, num_parallel_calls=4)
image_label_ds = image_label_ds.batch(BATCH_SIZE)

# Load
image_label_ds = image_label_ds.cache(filename='./cache.tf-image')

# Prefetch must be final Dataset in input pipeline
ds = ds.apply(
    tf.data.experimental.prefetch_to_device(device="/gpu:0", buffer_size=AUTOTUNE)) 

timeit(image_label_ds)